# Hosting 

In this tutorial we will create a docker container and deploy it to Heroku. The docker container will contain code from the previous tutorials and write data to your Clarify integration. 

## Prerequisites
It is highly recommended that you have gone through the [Introduction Notebook]() for the basics of using PyClarify. This tutorial is will not 

## What do you need
1. [Google Cloud Account](https://cloud.google.com/free)
2. [Google Cloud SDK](https://cloud.google.com/sdk/docs/install)
3. [Docker](https://www.docker.com/products/docker-desktop)

## What we will do
1. 
2. 
3. [Google Cloud Hosting](#hosting)

<hr>
Other resources:
- 
- 

## Getting started
The first step is downloading both [Docker](https://www.docker.com/products/docker-desktop) and [Google Cloud SDK](https://cloud.google.com/sdk/docs/install).

Once you have installed them both, we can start by creating the package we want to host. (ALEXIA'S STUFF COMES NEXT)

<a name="hosting"></a>
# Google Cloud Hosting
Now that we have a working docker container, we can push it to a cloud environment. We will use the Google Cloud Run environment to host our docker image. Google has great guides on [adding you docker image to their registry](https://cloud.google.com/container-registry/docs/pushing-and-pulling) and [deploying it on their run service](https://cloud.google.com/run/docs/deploying#command-line). However, for closing the loop we will provide a guide on how to use their system. 

> Keep in mind that their systems may change in the future and their guides should be the source of truth!

## Getting started
Now that we have a working docker container, we can push it to a cloud environment. As mentioned in the beginning we will be using Google Cloud to do this. Make sure you have downloaded [Google Cloud SDK](https://cloud.google.com/sdk/docs/install) before we start.

> Check that it is installed correctly by running `gcloud version`.

Before we connect to your google account in with the SDK, you can [create a project](https://console.cloud.google.com/projectcreate) so that we can connect to the project as well. 

> If you have multiple Google Accounts, make sure that you are logged into the correct one.

Once that is completed you can connect to your google account running `gcloud auth login` in the terminal. 

For the rest of this tutorial we will use `YOUR-PROJECT-ID` and `YOUR-IMAGE-TAG` as placeholders for what you called your newly created project in google cloud and what you tag your image with in docker. 

## Workflow
Hosting a docker image on google cloud is done in two steps:
1. Publishing the image
2. Pushing the published image to a run environment


### Publishing the image
For the first step we start by building the image and adding a `tag` to it:
```sh
$ docker build . --tag YOUR-IMAGE-TAG
```
Now that we have a built image with a tag, we can set its target path in [Container Registry](https://cloud.google.com/container-registry/docs/pushing-and-pulling#add-registry), including the gcr.io registry host and the project ID of `YOUR-PROJECT-ID`:
```sh
$ docker tag YOUR-IMAGE-TAG gcr.io/YOUR-PROJECT-ID/YOUR-IMAGE-TAG
```
The final step of publishing the image is to push it to the registry. To do this just run:
```sh
$ docker push gcr.io/YOUR-PROJECT-ID/YOUR-IMAGE-TAG
```
Now your image is published! The Container Registry adds the registry to your project, creates a storage bucket for the registry, and stores the image. For more information please refer to [their guides](https://cloud.google.com/container-registry/docs/pushing-and-pulling).

### Pushing the published image to a run environment
We will use the [Cloud Run](https://cloud.google.com/run) to run the image that we have published.

```sh
$ gcloud run deploy YOUR-IMAGE-TAG --image gcr.io/YOUR-PROJECT-ID/YOUR-IMAGE-TAG
```

After you run this command you will be asked three questions. They are pretty self explainatory, but if you want to follow completely run:
- 1 (enter)
- 13 (enter)
- y (enter)

> Google provides a quickstart guide for building and deploying python services [here](https://cloud.google.com/run/docs/quickstarts/build-and-deploy/python).

## Caveats
Google Cloud Run uses [autoscaling](https://cloud.google.com/run#all-features) which means that it scales from zero to N containers depending on the traffic recieved. This means that if your site recieves no traffic it will scale down to zero instances after 15 minutes and thus stop sending data with the background scheduler. As this containers main purpose is write to Clarify with a background scheduler, we want it to be only one container at all times. We can set this in [the console](https://cloud.google.com/run/docs/configuring/min-instances#setting) or with the command:

```sh
$ gcloud run deploy YOUR-IMAGE-TAG --image gcr.io/YOUR-PROJECT-ID/YOUR-IMAGE-TAG --min-instances 1 --max-instances 1
```